In [1]:
a = [1.0,2.0,1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0

In [4]:
a

[1.0, 2.0, 3.0]

In [1]:
import torch


In [17]:
a = torch.ones(3)   # 1 维
a

tensor([1., 1., 1.])

In [18]:
float(a[1])

1.0

In [20]:
a[1]

tensor(1.)

In [21]:
# 将list传给tensor，变为tensor
points = torch.tensor([4.0,1.0,5.0,3.0,2.0,1.0])   # 把坐标存在1D tensor
points

tensor([4., 1., 5., 3., 2., 1.])

In [24]:
float(points[0]),float(points[1])

(4.0, 1.0)

In [25]:
points = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [29]:
points.size()

torch.Size([3, 2])

In [ ]:
# named tensors

In [34]:
img_t = torch.randn(3,5,5)  #模仿图像的维度
weights = torch.tensor([0.2126,0.7152,0.0722])   # 每个通道的权重
# 构建有两个batch的tensor
batch_t = torch.randn(2,3,5,5)  

In [35]:
img_gray_naive = img_t.mean(-3)   # 通道维度上未加权的平均 即转换为灰度图像
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape,batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [36]:
unsqueezed_weights= weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t*unsqueezed_weights)
batch_weights = (batch_t*unsqueezed_weights)    #将每个通道里的像素都与权重相乘
img_gray_weighted = img_weights.sum(-3)        # 三通道变为单通道
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape,batch_t.shape,unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [37]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw',img_t,weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw',batch_t,weights) #以上操作的快捷方法 
batch_gray_weighted_fancy.shape   

torch.Size([2, 5, 5])

In [38]:
weights_named = torch.tensor([0.2126,0.7152,0.0722],names=['channels'])
weights_named

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [44]:
img_named =img_t.refine_names(...,'channels','rows','columns')
batch_named = batch_t.refine_names(...,'channels','rows','columns')

print('img named:',img_named.shape,img_named.names)
print('batch named:',batch_named.shape,batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [45]:
weights_aligned = weights_named.align_as(img_named)   # 自动补齐维度
weights_aligned.shape,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [46]:
gray_named = (img_named*weights_aligned).sum('channels')  # 一些函数也接受命名的维度作为参数
gray_named.shape,gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [47]:
# 取消维度命名

gray_plain = gray_named.rename(None)
gray_plain.shape,gray_plain.names

(torch.Size([5, 5]), (None, None))

In [49]:
# 指定张量的类型,使用参数

double_points = torch.ones(10,2,dtype = torch.double)
short_points = torch.tensor([[1,2],[3,4]],dtype = torch.short)

In [50]:
short_points.dtype


torch.int16

In [51]:
# 使用函数转换数值类型

double_points = torch.zeros(10,2).double()
short_points=torch.ones(10,2).short()

In [52]:
# 或者
double_points=torch.zeros(10,2).to(torch.double)
short_points = torch.ones(10,2).to(dtype=torch.short)

In [ ]:
# tensor API


In [62]:
a = torch.ones(3,2,5)
a_t = torch.transpose(a,0,2)    #指定维度的转置
a.shape,a_t.shape

(torch.Size([3, 2, 5]), torch.Size([5, 2, 3]))

In [64]:
a_t2 = a.transpose(1,2)
a_t2.shape

torch.Size([3, 5, 2])

### tensor 操作的分类

- 构建操作：构造张量的函数，比如 *ones* 和 *from_numpy*
- 索引、切片、连接、变形操作：改变shape,步长等函数，比如 *transpose*
- 数学操作
    
 

In [71]:
# 在存储的角度看张量

points =torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points.storage()   # 尽管张量是3行2列，但是被存储在一个连续的数组中，也就是说，存储总是1维的

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [72]:
points_storage = points.storage()   # 将points_storage 指向存储的points
points_storage[0]            # 手动索引

4.0

In [73]:
points_storage[0] = 11.0     #改变存储的值自然会改变tensor的值
points

tensor([[11.,  1.],
        [ 5.,  3.],
        [ 2.,  1.]])

In [75]:
points[2,1] = 50.0        # 改变tensor的值也会改变存储的值
points.storage() 

 11.0
 1.0
 5.0
 3.0
 2.0
 50.0
[torch.FloatStorage of size 6]

In [80]:
# 就地操作如   zeros_()  并不是新建一个tensor，而是在原地操作，改变原张量
# 任何不带下划线的方法都不会改变源张量，而是返回一个新的张量:


a = torch.ones(3, 2)

a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [78]:
a.storage()

 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
[torch.FloatStorage of size 6]

### Tensor metadata: Size, offset, and stride

<img src="https://raw.githubusercontent.com/liuzhaoo/markdown_pics/master/img/size,offset,stride.png" style="zoom:50%;" />

- size：第一个索引为行，第二个为列
- offset是存储中对应tensor第一个元素的索引（在此例中offset=1,因为tensor中第一个元素对应存储中1的索引的量）
- stride:是一个元组，第一个元素是列数，代表到下一行对应元素的距离；第二个元素代表到下一列的距离

>通过存储访问一个2D tensor的方式为： 
storage_offset +stride[0] \* i +stride[1] \* j

In [20]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()   # second_point 是一个张量，它在存储中的偏移是2

2

In [21]:
second_point.size()

torch.Size([2])

In [22]:
points.stride()

(2, 1)

In [23]:
second_point.storage_offset()   

2

In [30]:
second_point.storage()  # 此张量比原张量少了一维,但是还使用的原张量的存储

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [33]:
second_point.stride()

(1,)

In [34]:
# 改变子张量的元素会影响原张量
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [35]:
second_point

tensor([5., 3.])

In [36]:
second_point[0]=10.0

In [37]:
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [4]:
# 可以使用克隆的方式，新建一个tensor

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [5]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [6]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [7]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [8]:
id(points.storage()) == id(points_t.storage())  # 转置后的tensor与原tensor是同一个storage

True


tensor 在storage中的顺序是不变的，转置后的tensor  stride也在对应的维度上转置

<img src="https://raw.githubusercontent.com/liuzhaoo/markdown_pics/master/img/transpose.png" style="zoom:50%;" />

In [11]:
some_t = torch.randn(3,2, 3)

In [12]:
some_t.storage()

 0.490024596452713
 0.7767499685287476
 0.28726431727409363
 0.11238249391317368
 0.7568453550338745
 -0.23251444101333618
 1.0830928087234497
 -0.12701213359832764
 0.23480401933193207
 -0.13187548518180847
 -1.4970356225967407
 -0.832029402256012
 0.15464890003204346
 1.5715559720993042
 1.506649136543274
 0.006555233150720596
 0.2788148820400238
 0.49143603444099426
[torch.FloatStorage of size 18]

In [13]:
some_t

tensor([[[ 0.4900,  0.7767,  0.2873],
         [ 0.1124,  0.7568, -0.2325]],

        [[ 1.0831, -0.1270,  0.2348],
         [-0.1319, -1.4970, -0.8320]],

        [[ 0.1546,  1.5716,  1.5066],
         [ 0.0066,  0.2788,  0.4914]]])

In [14]:
# 高维tensor在storage中的顺序是按最右侧维度（即2维中的行里）连续存储的

In [15]:
# 有些操作只能在连续的tensor上运行，转置后的tensor是不连续的，对不连续的tensor操作不会产生任何结果

points.is_contiguous()

True

In [16]:
points_t.is_contiguous()

False

In [17]:
# 可以使用contiguous方法在非连续的tensor中新建一个连续的tensor

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [18]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [20]:
points_t.stride()

(1, 2)

In [21]:
points_t_con = points_t.contiguous()
points_t_con

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [22]:
points_t_con.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

In [23]:
points_t_con.stride()

(3, 1)

In [24]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [54]:
# 将张量放到gpu上
points_gpu =torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]],device = 'cuda:3')

In [55]:
# 将cpu上的tensor放到gpu
points_gpu2 = points_t_con.to(device ='cuda:1')

In [56]:
id(points_gpu.storage) == id(points_gpu2.storage)

True

In [38]:
id(points)

140277958830432

In [52]:
id(points_gpu.storage)

140277955974720

In [53]:
id(points_gpu2.storage)

140277954955536

In [57]:
points = 2*points

In [58]:
points_gpu = 2*points.to(device = 'cuda')

In [59]:
points_gpu += 4 

In [2]:
points_cpu = points_gpu.cpu()

NameError: name 'points_gpu' is not defined

In [3]:
# 与numpy的互相操作



points = torch.ones(3,4)
points_np = points.numpy()

In [4]:
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [5]:
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [7]:
points_np[0,0] = 10

In [8]:
points                    # 将tensor转换为numpy后，存储位置不变，因为是用的torch内的方法

tensor([[10.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]])

In [17]:
import numpy as np
a = np.zeros([3,3])

In [18]:
a

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [21]:
b = torch.tensor(a)

In [23]:
b[0,0] =10

In [27]:
a                      # 反之，将numpy数据转换为tensor，就是创建新的

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
# 但是使用pytorch内置方法可以共享存储区

In [28]:
points = torch.from_numpy(a)

In [31]:
a[0,0]=10

In [32]:
points

tensor([[10.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]], dtype=torch.float64)

In [36]:
# 保存张量

torch.save(points,'../data/p1ch3/my_points.t')

In [42]:
# 可以用文件描述符来代替文件名
with open('../data/p1ch3/my_points.t','wb') as f:
    torch.save(points,f)

In [44]:
# 加载
c = torch.load('../data/p1ch3/my_points.t')

In [47]:
# 或者用另一种方法

with open('../data/p1ch3/my_points.t','rb') as f:
    a = torch.load(f)

In [61]:
# 存到 h5py
import h5py
f = h5py.File('../data/p1ch3/my_points.hdf5','w')  # 新建一个文件
dset = f.create_dataset('coords',data = points.numpy())
f.close()


In [62]:
# HDF5 们可以在磁盘上索引数据集，并且只访问我们感兴趣的元素
# 文件打开时，不加载数据；只有在进行操作时才会加载
f = h5py.File('../data/p1ch3/my_points.hdf5','r')
dset = f['coords']          # coords  是一个关键字
last_points = dset[-2:]     # 可以使用索引操作

In [63]:
last_points      # 返回值是array,与numpy的类似

array([[0., 0., 0.],
       [0., 0., 0.]])

In [68]:
last_points = torch.from_numpy(dset[-2:])

In [69]:
last_points


tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [70]:
f.close

<bound method File.close of <HDF5 file "my_points.hdf5" (mode r)>>

In [110]:
# exercise

a = torch.tensor(range(9))  # 预测：size[9],offset=0,stride = (1,)


In [76]:
a.storage_offset()

0

In [77]:
a.shape

torch.Size([9])

In [78]:
a.stride()

(1,)

In [79]:
b = a.view(3,3)  # 转换为3，3  预测size[3,3],offset=0,stride=(3,1)

In [80]:
b.storage_offset()

0

In [82]:
b.stride()

(3, 1)

In [94]:
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [92]:
id(a.storage()) == id(b.storage())

True

In [93]:
c = b[1:,1:]   # 预测 offset  = 4 size[2,2] stride = (2,1)  预测错误
c

tensor([[4, 5],
        [7, 8]])

In [96]:
c.storage_offset()

4

In [98]:
c.size()

torch.Size([2, 2])

In [99]:
c.stride()      #  c海是在原来的storage，所以stride仍为（3，1）

(3, 1)

In [112]:
a = a.to(torch.float32)
a.sqrt()    # 不支持对long 、或short 操作

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])

### 本节有两个用到.to的地方

1. 转换数值类型


>`
double_points=torch.zeros(10,2).to(torch.double)
short_points = torch.ones(10,2).to(dtype=torch.short)
`

    等价于
>`
double_points = torch.zeros(10,2).double()
short_points=torch.ones(10,2).short()
`


2. 数据放到gpu或cpu

>`
points_gpu2 = points_t_con.to(device ='cuda:1')
`

    等价于
>`
points_gpu = points.cuda()
`